May: **Research Questions 2.1** (Ideal and Real)

# Which of the three upwelling mechanisms (time dependence, advection, convergence of isobaths, coastal trapped wave) provides the strongest contribution to upwelling in Mackenzie Canyon?

Where does upwelling occur?

In [1]:
import netCDF4 as nc
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import cmocean as cmo
import os,sys,fnmatch,time
import numpy as np
import matplotlib.gridspec as gridspec
from matplotlib import animation
from salishsea_tools.nc_tools import scDataset
from matplotlib import colors
from matplotlib import ticker
from matplotlib.colors import LinearSegmentedColormap

%matplotlib inline

import sys
sys.path.append('/ocean/imachuca/Canyons/analysis_mackenzie_canyon/notebooks/general_circulation/')
import general_functions
import quicklook

import warnings
warnings.filterwarnings("ignore")

# Functions

In [2]:
class MidpointNormalize(colors.Normalize):
    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        self.midpoint = midpoint
        colors.Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        result, is_scalar = self.process_value(value)
        (vmin,), _ = self.process_value(self.vmin)
        (vmax,), _ = self.process_value(self.vmax)
        resdat = np.asarray(result.data)
        result = np.ma.array(resdat, mask=result.mask, copy=False)
        x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
        res = np.interp(result, x, y)
        result = np.ma.array(res, mask=result.mask, copy=False)
        if is_scalar:
            result = result[0]
        return result

In [3]:
def get_salt_ud(dirname, fname, meshmaskname, dep_ind):
    
    y,x = slice(1,-1,None), slice(1,-1,None)
    time_ind = 24*9
    
    filesT = general_functions.get_files(dirname, fname, 'grid_T')
    filesU = general_functions.get_files(dirname, fname, 'grid_U')
            
    with scDataset(filesT) as dsT, scDataset(filesU) as dsU:
        deptht = dsT.variables['deptht'][:]
        vosaline0 = dsT.variables['vosaline'][:time_ind, dep_ind, y, x]
        sozotaux = dsU.variables['sozotaux'][:time_ind, 0, 0] 
        
    with nc.Dataset(os.path.join(dirname, meshmaskname), 'r') as dsM:
        tmask0 = dsM.variables['tmask'][0, dep_ind, y, x]
        
    tmask = np.tile(tmask0, (len(sozotaux), 1, 1, 1))  
    vosaline = np.ma.array(vosaline0, mask=1 - tmask)
    
    return vosaline, sozotaux, deptht, tmask[0, 0, ...]

In [4]:
def get_daily_vosaline(vosaline, tmask, days):
    
    # get daily salinity averages from hourly results
    vosaline_daily0 = quicklook.get_1day_avg_of_speeds(vosaline, days)
    tmask_new = np.tile(tmask, (vosaline_daily0.shape[0], 1, 1))
    vosaline_daily = np.ma.array(vosaline_daily0, mask=1 - tmask_new)
    
    return vosaline_daily

In [5]:
def get_daily_depth_ind(vosaline_daily, vosaline_ref_profile):
    
    depth_ind_daily = np.full_like(vosaline_daily, np.nan, dtype=np.int)
    
    # trace back the deptht_ind where this_vosaline occured the vosaline_ref     
    for d in range(depth_ind_daily.shape[-3]):
        for y in range(depth_ind_daily.shape[-2]):
            for x in range(depth_ind_daily.shape[-1]):
                this_vosaline = vosaline_daily[d, y, x]
                depth_ind_daily[d, y, x] = min(range(len(vosaline_ref_profile)), key=lambda i: abs(vosaline_ref_profile[i]-this_vosaline))
        print(d)

    return depth_ind_daily

In [6]:
def get_daily_depth_m(deptht, depth_ind_daily, tmask, days):

    # get deptht for deptht_ind of origin
    depth_m_daily0 = deptht[depth_ind_daily]
    tmask_new = np.tile(tmask, (days, 1, 1))
    depth_m_daily = np.ma.array(depth_m_daily0, mask=1 - tmask_new)
    
    return depth_m_daily

In [7]:
def plot_original_depth(depth_m_daily, tmask, deptht, dep_ind, levels, kind, case):
    cmap, norm = cmo.cm.delta, MidpointNormalize(midpoint=deptht[dep_ind])
    cmap.set_bad('silver')

    vmin, vmax = int(np.min(depth_m_daily)), int(np.max(depth_m_daily))
    xs = np.arange(tmask.shape[-1])
    ys = np.arange(tmask.shape[-2])

    fig, axes = plt.subplots(3, 3, figsize = (20, 21))
    for ax, n in zip(axes.flatten(), np.arange(9)):
        p = ax.pcolormesh(xs, ys, depth_m_daily[n, :, :], cmap=cmap, vmin=vmin, vmax=vmax, norm=norm)
        cs = ax.contour(xs, ys, depth_m_daily[n, :, :], levels = levels, colors='k', alpha=0.5)
        ax.clabel(cs, inline=1, fontsize=10, fmt ='%r')
        plt.setp(ax.get_xticklabels(), visible=False)
        plt.setp(ax.get_yticklabels(), visible=False)
        ax.tick_params(axis='both', which='both', length=0)
        ax.set_aspect(aspect='equal')
        ax.set_xlim([0, xs[-1]])
        ax.set_ylim([0, ys[-1]])
        ax.set_title('Day ' + str(n+1), fontsize=20)
        cbar = fig.colorbar(p, ax=ax, fraction=0.05, orientation='horizontal', pad=0.009)
        cbar.ax.tick_params(labelsize=13)
        tick_locator = ticker.MaxNLocator(nbins=6)
        cbar.locator = tick_locator
        cbar.update_ticks()

    fig.tight_layout(w_pad=1.2, h_pad=0.01)
    fig.suptitle(kind + ' - ' + case +': ' + 'original depth [m] for upwelling at depth = '+
                 str(int(deptht[dep_ind]))+' m', fontsize=25)
    plt.subplots_adjust(top=0.96)

    return fig

# Constants

In [8]:
vosaline_ref = nc.Dataset('/ocean/imachuca/Canyons/mackenzie_canyon/conditions/NEMO_files/salinity/salinity_for_agrif.nc')['vosaline'][:]
vosaline_ref_profile = vosaline_ref[0, :, 0, 0]

fname = '1_MCKNZ_1h_20170101_201701*'
meshmaskname = '1_mesh_mask.nc'

days = 9 
dep_inds = [1, 9]

# Ideal

In [9]:
kind = 'ideal'
case = 'base'

dirname = '/ocean/imachuca/Canyons/results_mackenzie/extended_domain/'+kind+'_'+case+'/'

levels = [125, 150]

In [10]:
for dep_ind, n in zip(dep_inds, np.arange(3)):
    vosaline, sozotaux, deptht, tmask = get_salt_ud(dirname, fname, meshmaskname, dep_ind)

    vosaline_daily = get_daily_vosaline(vosaline, tmask, days)

    depth_ind_daily = get_daily_depth_ind(vosaline_daily, vosaline_ref_profile)
    depth_m_daily = get_daily_depth_m(deptht, depth_ind_daily, tmask, days)
    
    fig = plot_original_depth(depth_m_daily, tmask, deptht, dep_ind, levels, kind, case)
    fig.savefig('../writing_images/depth_orig_'+kind+'_'+case+'_'+str(dep_ind)+'.png', dpi=100, bbox_inches='tight')
    plt.close(fig) 

0
1
2
3
4
5
6
7
8
0
1
2
3
4
5
6
7
8
